In [56]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve

data = pd.read_csv("cleaned_input/train_consolidated.csv")

In [57]:
data.head()
data = data.drop(["msno", "Unnamed: 0"], axis=1)
data.shape

(853753, 122)

In [58]:
#data = pd.DataFrame([[1,2,3,5,6,7,8,9],[1,3,9,7,5,4,5,7]], columns=['a','b','c','d','e','f','g','is_churn'])

In [59]:
model_data = data.sample(frac=.1)

In [60]:
X_train = model_data.drop('is_churn', axis=1);
#replaced infinite values with zero. MAKE SURE THIS IS the right thing to do 
from numpy import inf
X_train[X_train == -inf] = 0
X_train[X_train == inf] = 0
y_train = model_data['is_churn'];

In [61]:
X_train.replace([np.inf, -np.inf], 0)

,trans_count,date_min,date_max,date_count,num_25_sum,num_50_sum,num_75_sum,num_985_sum,num_100_sum,num_unq_sum,...,payment_method_id_encoded32,payment_method_id_encoded33,payment_method_id_encoded34,payment_method_id_encoded35,is_auto_renew_encoded0,is_auto_renew_encoded1,is_auto_renew_encoded2,is_cancel_encoded0,is_cancel_encoded1,is_cancel_encoded2
266403,4,20150103.0,20170228.0,10.0,415.0,171.0,135.0,121.0,1223.0,1838.0,...,1,0,0,0,0,1,0,0,1,0
488231,9,20150101.0,20170228.0,10.0,288.0,142.0,68.0,48.0,1874.0,2138.0,...,0,0,1,0,0,0,1,0,1,0
473452,19,20170301.0,20170331.0,1.0,139.0,34.0,31.0,27.0,945.0,982.0,...,0,0,1,0,0,0,1,0,1,0
465968,8,20170302.0,20170331.0,1.0,878.0,51.0,48.0,57.0,477.0,1368.0,...,0,0,0,0,0,0,1,0,1,0
561333,11,20160527.0,20170228.0,10.0,1745.0,326.0,201.0,307.0,3309.0,4558.0,...,0,0,0,1,0,0,1,0,1,0
703969,6,20170302.0,20170331.0,1.0,127.0,20.0,20.0,18.0,1047.0,1117.0,...,0,0,0,1,0,0,1,0,1,0
312055,25,20150101.0,20170227.0,10.0,1133.0,238.0,163.0,163.0,10275.0,5045.0,...,0,0,0,0,0,0,1,0,1,0
262955,16,20170306.0,20170331.0,1.0,16.0,1.0,3.0,2.0,1280.0,943.0,...,0,0,1,0,0,0,1,0,1,0
423106,27,20150101.0,20170228.0,10.0,3530.0,739.0,453.0,628.0,7523.0,11582.0,...,0,0,0,0,0,0,1,0,1,0
466846,23,20150101.0,20170228.0,10.0,2799.0,624.0,537.0,414.0,9978.0,10968.0,...,0,0,0,0,0,0,1,0,1,0


In [62]:
X_train.isnull().any()

trans_count                    False
date_min                       False
date_max                       False
date_count                     False
num_25_sum                     False
num_50_sum                     False
num_75_sum                     False
num_985_sum                    False
num_100_sum                    False
num_unq_sum                    False
total_secs_sum                 False
city                           False
bd                             False
registered_via                 False
registration_init_year         False
registration_init_month        False
registration_init_date         False
payment_method_id              False
payment_plan_days              False
plan_list_price                False
actual_amount_paid             False
is_auto_renew                  False
is_cancel                      False
discount                       False
is_discount                    False
amt_per_day                    False
membership_duration            False
m

In [63]:
print(np.all(np.isfinite(X_train)))
print(np.any(np.isnan(X_train)))

True
False


In [64]:
def plot_learning_curve(est, X_train, y_train):
    train_size, train_score, CV_score = learning_curve(est, X_train, y_train,train_sizes=np.linspace(0.1,1, 5));
    plt.figure();
    plt.plot(train_size, np.mean(train_score, axis=1), 'o-', color='b', label='training score');
    plt.plot(train_size, np.mean(CV_score, axis=1), 'o-', color='r', label='CV score');
    plt.xlabel("Training examples");
    plt.ylabel("Score");
    plt.fill_between(train_size, np.mean(train_score, axis=1) - np.std(train_score, axis=1),
                     np.mean(train_score, axis=1) + np.std(train_score, axis=1), alpha=0.1,
                     color="b");
    plt.fill_between(train_size, np.mean(CV_score, axis=1) - np.std(CV_score, axis=1),
                     np.mean(CV_score, axis=1) + np.std(CV_score, axis=1), alpha=0.1,
                     color="r");
    plt.title("Learning Curve", fontsize=20);
    plt.legend();
    plt.show();

In [65]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [66]:
#### making list of parameters for grid search
param_logit = {'C':[0.01,0.1,1,10,100], 'penalty':['l1','l2']}
param_knn = {'n_neighbors':[1,3,6,10], 'weights':['uniform','distance']}
param_svc = {'C':[0.01,0.1,1,10,100], 'kernel':{'rbf','sigmoid','precomputed'}}
#param_ann = {hidden_layer_sizes}
param_rfc = {'n_estimator':[5,10,15,20], 'max_features':[10,100,500,1000]}


In [67]:
model_list = {LogisticRegression():param_logit, KNeighborsClassifier():param_knn,
             SVC():param_svc, RandomForestClassifier():param_rfc};


In [68]:
for model,param in model_list.items():
    gscv = GridSearchCV(model, param);
    print("learning curve for\t{}".format(model))
    plot_learning_curve(gscv, X_train, y_train)

learning curve for	LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


KeyboardInterrupt: 